In [2]:
using Plots;
using DifferentialEquations;
using LinearAlgebra;
using Colors;

In [3]:
include("EulerCromer.jl")
include("RK4.jl")
include("lorentz_system.jl")
include("euler_method.jl")

EulerForward (generic function with 2 methods)

In [4]:
function SolutionLorenz(p, Δt, x₀, N, Solver)
    X = zeros(3,N+1)
    X[:,1] = x₀
    x = x₀
    for i = 2:N+1
        X[:,i] = Solver(p, X[:,i-1], Δt)
    end
    return X
end

function Points_on_Sphere(nr_of_dots, r)
    dots = zeros(3, nr_of_dots)
    for i=1:nr_of_dots
        dot = rand(-1.0:0.001:1.0,(1,3))
        dot = r*dot./norm(dot)
        dots[:,i] = dot
    end
    return dots
end


Points_on_Sphere (generic function with 1 method)

In [5]:
function PointsSolutions(p,Δt,N,Solver,points,displace=[0;0;0])
    solutions = []
    for i = 1:M
        x = points[:,i] + displace
        X = SolutionLorenz(p, Δt, x, N, Solver)
        push!(solutions, X)
    end
    return solutions
end

function PointsSolutionsJuliaSolver(p,Δt,N,Solver,points,displace=[0;0;0])
    solutions = []
    for i = 1:M
        x = points[:,i] + displace
        prob = ODEProblem(LorentzSystem,x,(0,Δt*N),[θ,μ,β])
        X = solve(prob, Solver)
        push!(solutions, X)
    end
    return solutions
end


PointsSolutionsJuliaSolver (generic function with 2 methods)

In [6]:

function PlotNucleusPlot(solutions)
    p = scatter(legend=false)
    for X = solutions
        scatter!(p, X[1,[1]], X[2,[1]], X[3,[1]], color=:blue)
        scatter!(p, X[1,[N]], X[2,[N]], X[3,[N]], color=:red)
        plot!(p, X[1,[1,N]], X[2,[1,N]], X[3,[1,N]], color=:black)
    end
    display(p)
end

function PlotTrajectories(solutions)
    p = plot()#legend=false)
    for X = solutions
        plot!(p, X[1,:], X[2,:], X[3,:]) #, color=:blue) 
    end
    display(p)
end

function PlotTrajectoriesInterpolated(solutions)
    p = plot()#legend=false)
    for X = solutions
        plot!(p, X, vars=(1,2,3)) #, color=:blue) 
    end
    display(p)
end

PlotTrajectoriesInterpolated (generic function with 1 method)

In [7]:
θ, μ, β = 10, 23.5, 8/3
param = [θ, μ, β]

val = sqrt(β*(μ-1))
a₀ = [0,0,0]
a₁ = [val, val, μ-1]
a₂ = [-val, -val, μ-1]

Δt = 0.001
N = 100000
x₀ = [1,-1,(2*(β*(μ-1)) + (μ+1)^2)/(2*(μ-1))]
# x₀ = a₁
# M = 100
# r = 3

3-element Vector{Float64}:
  1.0
 -1.0
 16.005555555555556

In [16]:
prob = ODEProblem(LorentzSystem,x₀,(0,Δt*N),param);
sol1 = solve(prob, RadauIIA5(), adaptive=false, dt=0.01);
sol2 = SolutionLorenz(param, Δt, x₀, N, RK4);

In [17]:
#plot(sol1, vars=(1,2,3))
length(sol1)


10001

In [ ]:
x₀ = a₀